# **Semantic Image Synthesis with Spatially-Adaptive Normalization**

**Authors: Taesung Park, Ming-Yu Liu, Ting-Chun Wang, Jun-Yan ZhuL**

**Original Paper**: https://arxiv.org/pdf/1903.07291.pdf

**Official Github**: https://github.com/NVlabs/SPADE

---

**Edited By Su Hyung Choi (Key Summary & Code Practice)**

If you have any issues on this scripts, please PR to the repository below.

**[Github: @JonyChoi - Computer Vision Paper Reviews]** https://github.com/jonychoi/Computer-Vision-Paper-Reviews

Edited Jan 16 2022

---

<img src="./imgs/figure1.png" />

### **Abstract**
<p>
We propose spatially-adaptive normalization, a simple
but effective layer for synthesizing photorealistic images
given an input semantic layout. Previous methods directly
feed the semantic layout as input to the deep network, which
is then processed through stacks of convolution, normalization, and nonlinearity layers. We show that this is suboptimal as the normalization layers tend to “wash away” semantic information. To address the issue, we propose using
the input layout for modulating the activations in normalization layers through a spatially-adaptive, learned transformation. Experiments on several challenging datasets
demonstrate the advantage of the proposed method over existing approaches, regarding both visual fidelity and alignment with input layouts. Finally, our model allows user
control over both semantic and style. Code is available at https://github.com/NVlabs/SPADE.
</p>


### **1. Introduction**
<p>
Conditional image synthesis refers to the task of generating photorealistic images conditioning on certain input data. Seminal work computes the output image by
stitching pieces from a single image (e.g., Image Analogies [16]) or using an image collection [7, 14, 23, 30, 35].
Recent methods directly learn the mapping using neural networks [3, 6, 22, 47, 48, 54, 55, 56]. The latter methods are
faster and require no external database of images.
</p>
<p>
    We are interested in a specific form of conditional image synthesis, which is converting a semantic segmentation
mask to a photorealistic image. This form has a wide range
of applications such as content generation and image editing [6, 22, 48]. We refer to this form as semantic image
synthesis. In this paper, we show that the conventional network architecture [22, 48], which is built by stacking convolutional, normalization, and nonlinearity layers, is at best arXiv:1903.07291v2 [cs.CV] 5 Nov 2019
suboptimal because their normalization layers tend to “wash
away” information contained in the input semantic masks.
To address the issue, we propose spatially-adaptive normalization, a conditional normalization layer that modulates the
activations using input semantic layouts through a spatiallyadaptive, learned transformation and can effectively propagate the semantic information throughout the network.
</p>
<p>
    We conduct experiments on several challenging datasets
including the COCO-Stuff [4, 32], the ADE20K [58], and
the Cityscapes [9]. We show that with the help of our
spatially-adaptive normalization layer, a compact network
can synthesize significantly better results compared to several state-of-the-art methods. Additionally, an extensive ablation study demonstrates the effectiveness of the proposed
normalization layer against several variants for the semantic
image synthesis task. Finally, our method supports multimodal and style-guided image synthesis, enabling controllable, diverse outputs, as shown in Figure 1. Also, please
see our SIGGRAPH 2019 Real-Time Live demo and try our
online demo by yourself.
</p>

### **2. Related Work**
<p>
<strong>Deep generative models</strong> can learn to synthesize images.
Recent methods include generative adversarial networks
(GANs) [13] and variational autoencoder (VAE) [28]. Our
work is built on GANs but aims for the conditional image
synthesis task. The GANs consist of a generator and a discriminator where the goal of the generator is to produce realistic images so that the discriminator cannot tell the synthesized images apart from the real ones.
</p>
<p>
<strong>Conditional image synthesis</strong> exists in many forms that differ in the type of input data. For example, class-conditional
models [3, 36, 37, 39, 41] learn to synthesize images given
category labels. Researchers have explored various models
for generating images based on text [18,44,52,55]. Another
widely-used form is image-to-image translation based on a
type of conditional GANs [20, 22, 24, 25, 33, 57, 59, 60],
where both input and output are images. Compared to
earlier non-parametric methods [7, 16, 23], learning-based
methods typically run faster during test time and produce
more realistic results. In this work, we focus on converting
segmentation masks to photorealistic images. We assume
the training dataset contains registered segmentation masks
and images. With the proposed spatially-adaptive normalization, our compact network achieves better results compared to leading methods.
</p>
<p>
<strong>Unconditional normalization layers</strong> have been an important component in modern deep networks and can be found
in various classifiers, including the Local Response Normalization in the AlexNet [29] and the Batch Normalization (BatchNorm) in the Inception-v2 network [21]. Other
popular normalization layers include the Instance Normalization (InstanceNorm) [46], the Layer Normalization [2],
the Group Normalization [50], and the Weight Normalization [45]. We label these normalization layers as unconditional as they do not depend on external data in contrast to
the conditional normalization layers discussed below.
</p>
<p>
<strong>Conditional normalization layers</strong> include the Conditional
Batch Normalization (Conditional BatchNorm) [11] and
Adaptive Instance Normalization (AdaIN) [19]. Both were
first used in the style transfer task and later adopted in various vision tasks [3, 8, 10, 20, 26, 36, 39, 42, 49, 54]. Different from the earlier normalization techniques, conditional normalization layers require external data and generally operate as follows. First, layer activations are normalized to zero mean and unit deviation. Then the normalized activations are denormalized by modulating the
activation using a learned affine transformation whose parameters are inferred from external data. For style transfer tasks [11, 19], the affine parameters are used to control
the global style of the output, and hence are uniform across
spatial coordinates. In contrast, our proposed normalization
layer applies a spatially-varying affine transformation, making it suitable for image synthesis from semantic masks.
Wang et al. proposed a closely related method for image
super-resolution [49]. Both methods are built on spatiallyadaptive modulation layers that condition on semantic inputs. While they aim to incorporate semantic information
into super-resolution, our goal is to design a generator for
style and semantics disentanglement. We focus on providing the semantic information in the context of modulating
normalized activations. We use semantic maps in different
scales, which enables coarse-to-fine generation. The reader
is encouraged to review their work for more details.
</p>


### **3. Semantic Image Synthesis**
<p>
Let m ∈ L
H×W be a semantic segmentation mask
where L is a set of integers denoting the semantic labels,
and H and W are the image height and width. Each entry
in m denotes the semantic label of a pixel. We aim to learn
a mapping function that can convert an input segmentation
mask m to a photorealistic image.
</p>
<p>
<table>
    <tbody>
        <tr>
            <td>
                <img src="./imgs/figure2.png" width="400" />
            </td>
        </tr>
    </tbody>
</table>
<strong>Spatially-adaptive denormalization.</strong> Let h
i denote the activations of the i-th layer of a deep convolutional network
for a batch of N samples. Let C
i be the number of channels in the layer. Let Hi
and Wi be the height and width
of the activation map in the layer. We propose a new conditional normalization method called the SPatially-Adaptive
(DE)normalization (1)
(SPADE). Similar to the Batch Normalization [21], the activation is normalized in the channelwise manner and then modulated with learned scale and
bias. Figure 2 illustrates the SPADE design. The activation value at site (n ∈ N, c ∈ C
i
, y ∈ Hi
, x ∈ Wi
) is
</p>
<table>
    <tbody>
        <tr>
            <td>
                <img src="./imgs/equation1.png" width="400" />
            </td>
        </tr>
    </tbody>
</table>
<p>
where h
i
n,c,y,x is the activation at the site before normalization and µ
i
c
and σ
i
c
are the mean and standard deviation of
the activations in channel c:
</p>
<table>
    <tbody>
        <tr>
            <td>
                <img src="./imgs/equation2.png" width="400" />
            </td>
            <td>
                <img src="./imgs/equation3.png" width="400" />
            </td>
        </tr>
    </tbody>
</table>
<p>
The variables γ
i
c,y,x(m) and β
i
c,y,x(m) in (1) are the
learned modulation parameters of the normalization layer.
In contrast to the BatchNorm [21], they depend on the input segmentation mask and vary with respect to the location
(y, x). We use the symbol γ
i
c,y,x and β
i
c,y,x to denote the
functions that convert m to the scaling and bias values at
the site (c, y, x) in the i-th activation map. We implement
the functions γ
i
c,y,x and β
i
c,y,x using a simple two-layer convolutional network, whose design is in the appendix.
</p>
<p>
In fact, SPADE is related to, and is a generalization
of several existing normalization layers. First, replacing
the segmentation mask m with the image class label and
making the modulation parameters spatially-invariant (i.e.,
γ
i
c,y1,x1 ≡ γ
i
c,y2,x2
and β
i
c,y1,x1 ≡ β
i
c,y2,x2
for any y1, y2 ∈
{1, 2, ..., Hi} and x1, x2 ∈ {1, 2, ..., Wi}), we arrive at the
form of the Conditional BatchNorm [11]. Indeed, for any
spatially-invariant conditional data, our method reduces to
the Conditional BatchNorm. Similarly, we can arrive at
the AdaIN [19] by replacing m with a real image, making the modulation parameters spatially-invariant, and setting N = 1. As the modulation parameters are adaptive to
the input segmentation mask, the proposed SPADE is better
suited for semantic image synthesis.
</p>
<table>
    <tbody>
        <tr>
            <td>
                <img src="./imgs/figure3.png" width="400" />
            </td>
        </tr>
    </tbody>
</table>
<p>
<strong>SPADE generator.</strong> With the SPADE, there is no need to
feed the segmentation map to the first layer of the generator, since the learned modulation parameters have encoded
enough information about the label layout. Therefore, we
discard encoder part of the generator, which is commonly
used in recent architectures [22, 48]. This simplification results in a more lightweight network. Furthermore, similarly
to existing class-conditional generators [36,39,54], the new
generator can take a random vector as input, enabling a simple and natural way for multi-modal synthesis [20, 60].
</p>
<p>
Figure 4 illustrates our generator architecture, which employs several ResNet blocks [15] with upsampling layers.
The modulation parameters of all the normalization layers
are learned using the SPADE. Since each residual block
operates at a different scale, we downsample the semantic
mask to match the spatial resolution.
</p>
<p>
We train the generator with the same multi-scale discriminator and loss function used in pix2pixHD [48] except that
we replace the least squared loss term [34] with the hinge
loss term [31,38,54]. We test several ResNet-based discriminators used in recent unconditional GANs [1, 36, 39] but
observe similar results at the cost of a higher GPU memory requirement. Adding the SPADE to the discriminator
also yields a similar performance. For the loss function, we
observe that removing any loss term in the pix2pixHD loss
function lead to degraded generation results.
</p>
<p>
<strong>Why does the SPADE work better?</strong> A short answer is that
it can better preserve semantic information against common
normalization layers. Specifically, while normalization layers such as the InstanceNorm [46] are essential pieces in
almost all the state-of-the-art conditional image synthesis
models [48], they tend to wash away semantic information
when applied to uniform or flat segmentation masks.
</p>
<img src="./imgs/figure4.png" />
<p>
Let us consider a simple module that first applies convolution to a segmentation mask and then normalization.
Furthermore, let us assume that a segmentation mask with
a single label is given as input to the module (e.g., all the
pixels have the same label such as sky or grass). Under this
setting, the convolution outputs are again uniform, with different labels having different uniform values. Now, after we
apply InstanceNorm to the output, the normalized activation
will become all zeros no matter what the input semantic label is given. Therefore, semantic information is totally lost.
This limitation applies to a wide range of generator architectures, including pix2pixHD and its variant that concatenates the semantic mask at all intermediate layers, as long
as a network applies convolution and then normalization to
the semantic mask. In Figure 3, we empirically show this is
precisely the case for pix2pixHD. Because a segmentation
mask consists of a few uniform regions in general, the issue
of information loss emerges when applying normalization.
</p>
<p>
    In contrast, the segmentation mask in the SPADE Generator is fed through spatially adaptive modulation without
normalization. Only activations from the previous layer are
normalized. Hence, the SPADE generator can better preserve semantic information. It enjoys the benefit of normalization without losing the semantic input information.
</p>
<p>
<strong>Multi-modal synthesis.</strong> By using a random vector as the
input of the generator, our architecture provides a simple
way for multi-modal synthesis [20, 60]. Namely, one can
attach an encoder that processes a real image into a random
vector, which will be then fed to the generator. The encoder
and generator form a VAE [28], in which the encoder tries
to capture the style of the image, while the generator combines the encoded style and the segmentation mask information via the SPADEs to reconstruct the original image. The
encoder also serves as a style guidance network at test time
to capture the style of target images, as used in Figure 1.
For training, we add a KL-Divergence loss term [28].
</p>

---
(1) Conditional normalization [11, 19] uses external data to denormalize
the normalized activations; i.e., the denormalization part is conditional.


### **4. Experiments**
<p>
<strong>Implementation details.</strong> We apply the Spectral Norm [38]
to all the layers in both generator and discriminator. The learning rates for the generator and discriminator are
0.0001 and 0.0004, respectively [17]. We use the ADAM
solver [27] with β1 = 0 and β2 = 0.999. All the experiments are conducted on an NVIDIA DGX1 with 8 32GB
V100 GPUs. We use synchronized BatchNorm, i.e., these
statistics are collected from all the GPUs.
</p>
<strong>Datasets.</strong> We conduct experiments on several datasets.
<ul>
<li>COCO-Stuff [4] is derived from the COCO dataset [32].
It has 118, 000 training images and 5, 000 validation images captured from diverse scenes. It has 182 semantic
classes. Due to its vast diversity, existing image synthesis models perform poorly on this dataset.</li>
<li>ADE20K [58] consists of 20, 210 training and 2, 000 validation images. Similarly to the COCO, the dataset contains challenging scenes with 150 semantic classes.</li>
<li>ADE20K-outdoor is a subset of the ADE20K dataset that
only contains outdoor scenes, used in Qi et al. [43].</li>
<li>Cityscapes dataset [9] contains street scene images in
German cities. The training and validation set sizes are
3, 000 and 500, respectively. Recent work has achieved
photorealistic semantic image synthesis results [43, 47]
on the Cityscapes dataset.</li>
<li>Flickr Landscapes. We collect 41, 000 photos from
Flickr and use 1, 000 samples for the validation set. To
avoid expensive manual annotation, we use a well-trained
DeepLabV2 [5] to compute input segmentation masks.
</li>
</ul>
<p>
We train the competing semantic image synthesis methods
on the same training set and report their results on the same
validation set for each dataset.
</p>
<img src="./imgs/figure5.png" />
<img src="./imgs/figure6.png" />
<img src="./imgs/table1.png" />
<p>
Performance metrics. We adopt the evaluation protocol
from previous work [6, 48]. Specifically, we run a semantic segmentation model on the synthesized images and compare how well the predicted segmentation mask matches the
ground truth input. Intuitively, if the output images are realistic, a well-trained semantic segmentation model should
be able to predict the ground truth label. For measuring the
segmentation accuracy, we use both the mean Intersection-over-Union (mIoU) and the pixel accuracy (accu). We use
the state-of-the-art segmentation networks for each dataset:
DeepLabV2 [5, 40] for COCO-Stuff, UperNet101 [51] for
ADE20K, and DRN-D-105 [53] for Cityscapes. In addition to the mIoU and the accu segmentation performance
metrics, we use the Frechet Inception Distance (FID) [ ´ 17]
to measure the distance between the distribution of synthesized results and the distribution of real images.
</p>
<p>
<strong>Baselines.</strong> We compare our method with 3 leading semantic image synthesis models: the pix2pixHD model [48],
the cascaded refinement network (CRN) [6], and the semiparametric image synthesis method (SIMS) [43]. The
pix2pixHD is the current state-of-the-art GAN-based conditional image synthesis framework. The CRN uses a deep
network that repeatedly refines the output from low to high
resolution, while the SIMS takes a semi-parametric approach that composites real segments from a training set and
refines the boundaries. Both the CRN and SIMS are mainly
trained using image reconstruction loss. For a fair comparison, we train the CRN and pix2pixHD models using the
implementations provided by the authors. As image synthesis using the SIMS requires many queries to the training dataset, it is computationally prohibitive for a large dataset
such as the COCO-stuff and the full ADE20K. Therefore,
we use the results provided by the authors when available.
</p>
<img src="./imgs/figure7.png" />
<p>
<strong>Quantitative comparisons.</strong> As shown in Table 1, our
method outperforms the current state-of-the-art methods by
a large margin in all the datasets. For the COCO-Stuff, our
method achieves an mIoU score of 35.2, which is about 1.5
times better than the previous leading method. Our FID
is also 2.2 times better than the previous leading method.
We note that the SIMS model produces a lower FID score
but has poor segmentation performances on the Cityscapes
dataset. This is because the SIMS synthesizes an image by
first stitching image patches from the training dataset. As
using the real image patches, the resulting image distribution can better match the distribution of real images. However, because there is no guarantee that a perfect query (e.g.,
a person in a particular pose) exists in the dataset, it tends
to copy objects that do not match the input segments.
</p>
<p>
<strong>Qualitative results.</strong> In Figures 5 and 6, we provide qualitative comparisons of the competing methods. We find that
our method produces results with much better visual quality
and fewer visible artifacts, especially for diverse scenes in
the COCO-Stuff and ADE20K dataset. When the training
dataset size is small, the SIMS model also renders images
with good visual quality. However, the depicted content
often deviates from the input segmentation mask (e.g., the
shape of the swimming pool in the second row of Figure 6).
</p>
<table>
    <tbody>
        <tr>
            <td>
                <img src="./imgs/table2.png" width="400" />
            </td>
        </tr>
    </tbody>
</table>
<p>
In Figures 7 and 8, we show more example results from
the Flickr Landscape and COCO-Stuff datasets. The proposed method can generate diverse scenes with high image
fidelity. More results are included in the appendix.
</p>
<p>
<strong>Human evaluation.</strong> We use the Amazon Mechanical Turk
(AMT) to compare the perceived visual fidelity of our
method against existing approaches. Specifically, we give
the AMT workers an input segmentation mask and two
synthesis outputs from different methods and ask them to
choose the output image that looks more like a corresponding image of the segmentation mask. The workers are given
unlimited time to make the selection. For each comparison,
we randomly generate 500 questions for each dataset, and
each question is answered by 5 different workers. For quality control, only workers with a lifetime task approval rate
greater than 98% can participate in our study.
</p>
<img src="./imgs/figure8.png" />
<table>
    <tbody>
        <tr>
            <td>
                <img src="./imgs/table3.png" width="400" />
            </td>
        </tr>
    </tbody>
</table>
<p>
Table 2 shows the evaluation results. We find that users strongly favor our results on all the datasets, especially on
the challenging COCO-Stuff and ADE20K datasets. For the
Cityscapes, even when all the competing methods achieve
high image fidelity, users still prefer our results.
</p>
<p>
<strong>Effectiveness of the SPADE.</strong> For quantifying importance
of the SPADE, we introduce a strong baseline called
pix2pixHD++, which combines all the techniques we find
useful for enhancing the performance of pix2pixHD except
the SPADE. We also train models that receive the segmentation mask input at all the intermediate layers via feature
concatenation in the channel direction, which is termed as
pix2pixHD++ w/ Concat. Finally, the model that combines the strong baseline with the SPADE is denoted as
pix2pixHD++ w/ SPADE.
</p>
<table>
    <tbody>
        <tr>
            <td>
                <img src="./imgs/table4.png" width="400" />
            </td>
        </tr>
    </tbody>
</table>
<p>
As shown in Table 3, the architectures with the proposed
SPADE consistently outperforms its counterparts, in both
the decoder-style architecture described in Figure 4 and
more traditional encoder-decoder architecture used in the
pix2pixHD. We also find that concatenating segmentation
masks at all intermediate layers, a reasonable alternative
to the SPADE, does not achieve the same performance as
SPADE. Furthermore, the decoder-style SPADE generator
works better than the strong baselines even with a smaller
number of parameters.
</p>
<img src="./imgs/figure9.png" />
<p>
<strong>Variations of SPADE generator.</strong> Table 4 reports the performance of several variations of our generator. First, we
compare two types of input to the generator where one is the
random noise while the other is the downsampled segmentation map. We find that both of the variants render similar
performance and conclude that the modulation by SPADE
alone provides sufficient signal about the input mask. Second, we vary the type of parameter-free normalization layers before applying the modulation parameters. We observe
that the SPADE works reliably across different normalization methods. Next, we vary the convolutional kernel size
acting on the label map, and find that kernel size of 1x1
hurts performance, likely because it prohibits utilizing the
context of the label. Lastly, we modify the capacity of the
generator by changing the number of convolutional filters.
We present more variations and ablations in the appendix.
</p>
<p>
<strong>Multi-modal synthesis.</strong> In Figure 9, we show the multimodal image synthesis results on the Flickr Landscape
dataset. For the same input segmentation mask, we sample different noise inputs to achieve different outputs. More
results are included in the appendix.
</p>
<p>
<strong>Semantic manipulation and guided image synthesis.</strong> In
Figure 1, we show an application where a user draws different segmentation masks, and our model renders the corresponding landscape images. Moreover, our model allows
users to choose an external style image to control the global
appearances of the output image. We achieve it by replacing the input noise with the embedding vector of the style
image computed by the image encoder.
</p>

### **5. Conclusion**
<p>
We have proposed the spatially-adaptive normalization,
which utilizes the input semantic layout while performing
the affine transformation in the normalization layers. The
proposed normalization leads to the first semantic image
synthesis model that can produce photorealistic outputs for
diverse scenes including indoor, outdoor, landscape, and
street scenes. We further demonstrate its application for
multi-modal synthesis and guided image synthesis.
</p>
<p>
<strong>Acknowledgments.</strong> We thank Alexei A. Efros, Bryan
Catanzaro, Andrew Tao, and Jan Kautz for insightful advice. We thank Chris Hebert, Gavriil Klimov, and Brad
Nemire for their help in constructing the demo apps. Taesung Park contributed to the work during his internship at
NVIDIA. His Ph.D. is supported by a Samsung Scholarship.
</p>